In [1]:
import json
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
m0_logs1 = json.load(open("combined_logs_m0_prompt_k_1_15_index_0_200.json", "r", encoding="utf-8"))
m0_logs2 = json.load(open("combined_logs_m0_prompt_k_1_15_index_200_400.json", "r", encoding="utf-8"))
m0_logs3 = json.load(open("combined_logs_m0_prompt_k_1_15_index_400_420.json", "r", encoding="utf-8"))
m0_logs4 = json.load(open("combined_logs_m0_prompt_k_1_15_index_420_600.json", "r", encoding="utf-8"))

In [11]:
m0_logs = m0_logs1["logs"] + m0_logs2["logs"] + m0_logs3["logs"] + m0_logs4["logs"]

In [15]:
m0_stats = {
    "pass_at_k": {i: 0.0 for i in range(1, 16)},
    "pass_at_k_prune": {i: 0.0 for i in range(1, 16)},
}

In [16]:
for benchmark in m0_logs:
    for k in range(1, 16):
        m0_stats["pass_at_k"][k] += benchmark["pass_at_k"][str(k)]
        m0_stats["pass_at_k_prune"][k] += benchmark["pass_at_k_prune"][str(k)]